## **Interfaces**

---
The word interface has several meanings in programming. Often you hear people referring to an interface when they are actually talking about an Application Programming Interface, an API. This idea of an interface is the idea of a shared boundary between parts of a program, or even users and a program, for instance a GUI is also an interface, it is a Graphical User Interface. API and GUIs are not the type of interface we are discussing here.   

In object-oriented design an interface has a different meaning. It is programming by contract (Betrand Meyer, https://en.wikipedia.org/wiki/Design_by_contract), an interface is a collection of methods signatures e.g., `def signature(self, message: str)->str`, and sometimes attributes, that functions as a contract. It states that if you implement this interface then the class is of a certain type or subtype. 

Python knows three types of interfaces:   
 1. Duck Typing
 2. Abstract Base Classes 
 3. Static Protocols    

In this notebook I will only discuss the first two. The latter needs to be used in conjunction with an external type checker like MyPy, and is not interesting for our discussion. The interfaces we do discuss are integral part of OO-programming/design and are must know for anyone who wishes to program in the object-oriented paradigm. 

If you were a bit confused with my somewhat opaque description of an interface, a simple example of Duck Typing will lift the veil.


#### **DuckTyping** 
DuckTyping is the first kind of interface. it is the oldest interface in Python, and it is probably still most used. DuckTyping as an interface is a convention, it something programmers agree to. People who program in Python agree that DuckTyping is an interface, not that it actually is an interface, it is a class with unimplemented method (signatures), that sort of functions as an interface.

In my opinion it is typical Python solution, flexible, easy to grasp. Unfortunately, impossible to enforce and prone to faults, but that is also Python. 

For more information on DuckTyping see https://en.wikipedia.org/wiki/Duck_typing


In [2]:
from decimal import Decimal

class BankAccount:
    '''a duck typed bankaccount interface'''
    
    def deposit(self, amount:float)->None:
        ...
    
    def withdrawl(self, amount:float)->None:
        ...
    
    def transfer(self, counter:str, amount)->str:
        ...
    
    def get_balance(self)->str:
        ...
    

A simple DuckTyped bank account interface. As you can see all the methods have a signature, but no other body than pass. I need the ellipsis for not writing a true interface but one by convention, technically (as the keyword says) `BankAccount` is a class. Python has no separate concept of an interface. It knows an object (everything is an object) and everything has a type.

The class functions here as an interface, subclass the interface and you have kind of a subtype. I will have to be incredibly careful with the wording here for Python lacks the ability distinct type and subtype, it can only make a distinction between base class and subclass. Which is why Pythonista's confuse the two often.

The definitions of the two concepts are:
 * Type: A type is a set of values, equipped with one or more operations that can be applied uniformly to all these values
 * Class: A class is a set of similar objects. All objects of a class have similar components and are equipped with the same operations.
    
As you can see from these definitions, types and classes are nearly the same but there is a subtle difference. Type is related to and derived from set theory. We consider a set of values, for instance all integers $\mathbb{Z}$ and the operations from $\mathbb{Z} \rightarrow \mathbb{Z}$ defined for it, such as $ +, -, \times$. 

A class is obviously built upon this, but more limited in the sense that you would have to name the variables (I dare you to do this in $\mathbb{Z}$) and freer, as you can pretty much define any method you want to on a class. Furthermore, you cannot recursively define classes, which you can do for types.

This is complicated subject, I just touched on it, because, if nothing else, you should understand there is a difference between the two concepts. For more information on types see https://en.wikipedia.org/wiki/Type_system. 


In [4]:
class CurrentAccount(BankAccount):
    '''current_account implements the BankAccount interface'''
    
    def __init__(self, account_nr:str, account_holders:list[str],balance:float=0,overdraft:float=0):
        self.account_nr      = account_nr
        self.account_holders = account_holders
        self.balance         = Decimal(str(balance))
        


In [5]:
acc = CurrentAccount(account_nr='NL57SNSB0987654', account_holders=['George'])
print(acc)

In [6]:
acc.account_nr

'NL57SNSB0987654'

It works as expected, now a DuckTyped interface is just a class so I have inherited the methods of the superclass, they just wont do anything.

In [9]:
acc.get_balance()

In [10]:
acc.deposit(500)

I will need to implement the methods (obviously so of course).

In [11]:
class CurrentAccount(BankAccount):
    '''current_account implements the BankAccount interface'''
    
    def __init__(self, account_nr:str, account_holders:list[str],balance:float=0,overdraft:float=0):
        self.account_nr      = account_nr
        self.account_holders = account_holders
        self.balance         = Decimal(str(balance))
        
    def deposit(self, amount:float)->None:
        self.balance = self.balance = Decimal(str(amount))
    
    def withdrawl(self, amount:float)->None:
        self.balance -= Decimal(str(amount))

    def transfer(self, counter:str, amount)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            return f'the sum of {amount} euro has been transferred to account_nr {counter}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer'
    
    def get_balance(self)->str:
        return f'current balance is {self.balance} euro.'      
    
    #this a special method that allows you to bring a user friendly print to the screen
    def __str__(self)->str:
        return f'the account with number {self.account_nr} has a balance of {self.balance} euro.' 
    
    def set_overdraft(self, amount)->None:
        self.overdraft += Decimal(str(amount))
    

In [12]:
acc = CurrentAccount(account_nr='NL57SNSB0987654', account_holders=['George'])
print(acc)

the account with number NL57SNSB0987654 has a balance of 0 euro.


In [13]:
acc.deposit(57800)
print(acc)

the account with number NL57SNSB0987654 has a balance of 57800 euro.


In [14]:
repr(acc)

'<__main__.CurrentAccount object at 0x00000181847D7590>'

In [15]:
acc.get_balance()

'current balance is 57800 euro.'

**Object representations**     
As you know, Python has two type of object representations:
 1. `__repr()__` which contains information useful to programmers.
 2. `__str()__`  which contains information useful to users.

The first time we created acc and printed it we got <__main__.Current_account object at 0x000001F42A281850> this is not very useful information. I therefor created a useful message to go with a print of the object for users.    

Unfortunately there is no useful information for the fellow programmer `repr(acc)` $\rightarrow$ `<__main__.Current_account object at 0x000001F42A22C1D0>` let me make that useful to by overriding the `__repr__()` method 

In [16]:
class CurrentAccount(BankAccount):
    '''Currentaccount implements the BankAccount interface'''
    
    def __init__(self, account_nr:str, account_holders:list[str],balance:float=0,overdraft:float=0):
        self.account_nr      = account_nr
        self.account_holders = account_holders
        self.balance         = Decimal(str(balance))
        
    def deposit(self, amount:float)->None:
        self.balance += Decimal(str(amount))
    
    def withdrawl(self, amount:float)->None:
        self.balance -= Decimal(str(amount))

    def transfer(self, counter:str, amount:float)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            return f'the sum of {amount} euro has been transferred to account_nr {counter}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer'
    
    def get_balance(self)->str:
        return f'current balance is {self.balance} euro.'    
    
    def set_overdraft(self, amount)->None:
        self.overdraft += Decimal(str(amount))
        
    #special methods
    def __str__(self)->str:
        return f'the account with number {self.account_nr} has a balance of {self.balance} euro.' 
    
    def __repr__(self)->str:
        return f'Currentaccount(account_nr={self.account_nr}, account_holders={self.account_holders}, balance={self.balance})' 
    


We have useful information for the programmer.    

Two quick sidenotes:
 1. If you type in an object in the notebook console you get repr function back, not the print.
 2. If you want to use the `repr` in a f-string you need to write !r in the placeholder; like in `f'bla {attr !r} bla'`.

In [17]:
acc = CurrentAccount(account_nr='NL57SNSB0987654', account_holders=['George'])
acc

Currentaccount(account_nr=NL57SNSB0987654, account_holders=['George'], balance=0)

Let me create a few more classes that implement the BankAccount interface, just so you get a feel for it.

In [18]:
class SavingsAccount(BankAccount):
    '''a savings account class'''
    #class variable 
    interest_rate:Decimal=Decimal('0')
    
    def __init__(self, account_nr:str, account_holders:list[str], counter_account:str, balance:float=0):
        self.account_nr      = account_nr
        self.account_holders = account_holders
        self.counter_account = counter_account
        self.balance         = Decimal(str(balance))
    
    def deposit(self, amount:float)->None:
        self.balance += Decimal(str(amount))
    
    def transfer(self, amount:float)->str:
        amount = Decimal(str(amount))
        if self.balance + self.overdraft >= amount:
            return f'the sum of {amount} euro has been transferred to account_nr {self.counter_account}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer'
    
    def get_balance(self)->str:
        return f'current balance is {self.balance} euro.'
    
    @classmethod
    def set_interest_rate(self, rate:float)->None:
        self.interest_rate = Decimal(str(rate))        
    
    #special methods
    def __str__(self)->str:
        return f'the savings account with number {self.account_nr} has a balance of {self.balance} euro.' 
    
    def __repr__(self)->str:
        return f'{self.__class__.__name__}(account_nr={self.account_nr}, account_holders={self.account_holders},counter_account={self.counter_account}, balance={self.balance},interest_rate={self.interest_rate})' 
    

In [19]:
sav = SavingsAccount(account_nr='NL57SNSB1234567', account_holders=['George','Croc'], counter_account='NL57SNSB0987654')
sav.deposit(78000)
print(sav)

the savings account with number NL57SNSB1234567 has a balance of 78000 euro.


In [20]:
sav

SavingsAccount(account_nr=NL57SNSB1234567, account_holders=['George', 'Croc'],counter_account=NL57SNSB0987654, balance=78000,interest_rate=0)

In [21]:
sav.set_interest_rate(2.5)
sav

SavingsAccount(account_nr=NL57SNSB1234567, account_holders=['George', 'Croc'],counter_account=NL57SNSB0987654, balance=78000,interest_rate=2.5)

In [22]:
sav2 = SavingsAccount(account_nr='NL57SNSB121212', account_holders=['Ente','Rhino'], counter_account='NL57SNSB09875555')
sav2

SavingsAccount(account_nr=NL57SNSB121212, account_holders=['Ente', 'Rhino'],counter_account=NL57SNSB09875555, balance=0,interest_rate=2.5)

There are a few noteworthy things in the SavingsAccount class:
 1. not all the methods are implemented, i.e., there is no withdrawal method, and no overdraft attribute. You can partially implement a DuckTyped interface.*  
 2. Also we have overridden a method we want one less argument.
 3. We introduced a class method (remember a class method is similar but not the same as a static method).    
 
 *This in my mind leads to a series of questions; if I partially implement an interface is it than still a subtype of that interface? Or is it a partial type? Does such a thing exist? Pyton has no way to establish subtype just subclass, it is definitely a subclass according to Python.

In [23]:
issubclass(SavingsAccount, BankAccount)

True

In [24]:
isinstance(SavingsAccount, BankAccount)

False

#### **Not so SOLID** (sidenote)
This might seem a trivial problem to you, but it has severe implications in programming practice. It is the reason why in Python you cannot program SOLID as you cannot perform a Liskov substitution aka strong behavioural subtyping. If I can only partially implement an interface, I cannot guarantee that I could use a subtype where I expect the supertype, the supertype may need implementations I do not have in my subtype.

Formally Liskov substitution states; if S subtypes T, what holds for T-objects holds for S-objects. Which is obviously not true in our case for I can withdraw from a BankAccount object but not from a SavingsAccount object.  

Is this sidenote important? Well not really but if someone tells you to program SOLID in Python, you might ask them how Python guarantees type? By now you should start to realize that Python does not.

In [25]:
from typing import NamedTuple

class Stock(NamedTuple):
    company:str
    current_price:Decimal = Decimal('0') 
    
class Fund(NamedTuple):
    name:str
    risc_factor:str
    invested_amount:Decimal=Decimal('0')
    fund_yield:Decimal=Decimal('0') 
    

In [26]:
asml = Stock(company='ASML',current_price=125)
asml

Stock(company='ASML', current_price=125)

In [27]:
microsoft = Stock(company='Microsoft',current_price=227)
microsoft

Stock(company='Microsoft', current_price=227)

In [28]:
portx = Fund(name='PORTX', risc_factor='medium', invested_amount=420000, fund_yield=10.7)
portx

Fund(name='PORTX', risc_factor='medium', invested_amount=420000, fund_yield=10.7)

In [29]:
tbwax = Fund(name='TBWAX', risc_factor='low', invested_amount=1000000, fund_yield=5.2)
tbwax

Fund(name='TBWAX', risc_factor='low', invested_amount=1000000, fund_yield=5.2)

In [30]:
class InvestmentAccount(BankAccount):
    '''an investment account class'''
    def __init__(self, account_nr:str, account_holders:list[str], counter_account:str, balance:float=0,stocks:list[(Stock,int,float)]=[], funds:list[Fund]=[]):
        self.account_nr      = account_nr
        self.account_holders = account_holders
        self.counter_account = counter_account
        self.balance         = Decimal(str(balance))
        self.stocks          = stocks
        self.funds           = funds
    
    def deposit(self, amount:float)->None:
        self.balance += Decimal(str(amount))
    
    def transfer(self, amount:float)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            return f'the sum of {amount} euro has been transferred to account_nr {self.counter_account}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer would you like to sell some stock or investment funds?'
    
    def get_balance(self)->str:
        return f'current balance is {self.balance} euro.'
    
    def buy_stock(self,stock:Stock, number:int)->str:
        if self.balance >= stock.current_price*number:
            self.balance -= stock.current_price*number
            self.stocks.append((Stock,number,stock.current_price))
            return f'we have added {number} of stock {stock.company} to your investment account at a price of {stock.current_price*number} euro.'
        else:
            return 'you have insufficient funds the transaction can`t be executed'
        
    def sell_stock(self,stock:str)->None:
        pass
    
    def invest_fund(self, fund:Fund, amount:float)->None:
        pass
    
    def disinvest_fund(self,fund_name:str)->None:
        pass
    
    #special methods
    def __str__(self)->str:
        return f'the investment account with number {self.account_nr} has a balance of {self.balance} euro.' 
    
    def __repr__(self)->str:
        return f'Investmentaccount(account_nr={self.account_nr}, account_holders={self.account_holders},counter_account={self.counter_account}, balance={self.balance},stocks={self.stocks}, funds={self.funds})' 
    
    

In [31]:
inv = InvestmentAccount(account_nr='KEMPEN753197', account_holders=['Ente','Rhino'],counter_account='NL19INSBEAUF998999')
inv

Investmentaccount(account_nr=KEMPEN753197, account_holders=['Ente', 'Rhino'],counter_account=NL19INSBEAUF998999, balance=0,stocks=[], funds=[])

In [32]:
inv.deposit(250000000)
inv.get_balance()

'current balance is 250000000 euro.'

In [33]:
print(inv)

the investment account with number KEMPEN753197 has a balance of 250000000 euro.


In [34]:
inv.buy_stock(stock=asml, number=5000)

'we have added 5000 of stock ASML to your investment account at a price of 625000 euro.'

In [32]:
inv.get_balance()

'current balance is 249375000 euro.'

As an exercise play with this class I have left you with some skeleton methods, implement these.

Can you use inheritance in combination with an interface implementation in Python? The answer is yes, Python supports multi class inheritance and our DuckTyped interface is just a class, subclassing a DuckTyped interface together with a regular class is quite common in Python. 

Multiple inheritance can lead to confusion and should be handled with care, see the inheritance notebook for more details. However, if I use multiple inheritance and both super classes have a similar method, the interpreter will use the Method Resolution Order to determine which method to use, and as such prevents much of potential problems. 

In the Objects&Classes notebook I showed you how you could use dataclasses as a framework for creating a class. We can use dataclasses here too, there is no influence on the inheritance relationship. The dataclass decorator is a closure, not a superclass. See the decorators notebook.


In [35]:
from dataclasses import dataclass, field
from typing import ClassVar

@dataclass
class SavingsAccount(BankAccount):
    '''a savings account class with a dataclass decorator'''
    account_nr:str
    counter_account:str
    account_holders:list[str] = field(default_factory=list) 
    balance:Decimal=Decimal('0')
    # class variable
    interest_rate:ClassVar[Decimal]=None
    
    def deposit(self, amount:float)->None:
        self.balance += Decimal(str(amount))
    
    def transfer(self, amount:float)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            return f'the sum of {amount} euro has been transferred to account_nr {self.counter_account}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer'
    
    def get_balance(self)->str:
        return f'current balance is {self.balance} euro.'
    
    def set_interest_rate(self, rate:float)->None:
        self.interest_rate = Decimal(str(rate))
    
    #special methods
    def __str__(self)->str:
        return f'the {self.__class__.__name__} with number {self.account_nr} has a balance of {self.balance} euro.' 

In [36]:
sav = SavingsAccount(account_nr='NL57SNSB1234567', account_holders=['George','Croc'], counter_account='NL57SNSB0987654')
sav

SavingsAccount(account_nr='NL57SNSB1234567', counter_account='NL57SNSB0987654', account_holders=['George', 'Croc'], balance=Decimal('0'))

In [37]:
print(sav)

the SavingsAccount with number NL57SNSB1234567 has a balance of 0 euro.


Again, we have a few noteworthy things:
 1. I didn't write an `__init__()` as is the habit.
 2. The `__init__()` from the `@dataclass` knows which fields to use.
 3. `field(default_factory=list)` can be used to specify fields with mutable default values such as lists, sets etc. This comes in place of `[]`.
 4. We can use class variables with dataclasses. A class variable belongs to the class and not the object, a class variable's lifespan is the programming running time. 
 5. I have used dataclasses version of `__repr__()` while implementing my own version of `__str__()`.

In [38]:
sav.set_interest_rate(2.7)
sav.interest_rate

Decimal('2.7')

However, I left an unclear print for interest rate, we should want a return value that is legible for the dimmest audience.    

I'll let you implement that as an exercise.

There is a lot of room to better this class, for instance we might want to use attribute validation here. This is an important subject, and we will discuss it in a separate notebook called Objects in Python.

#### **Abstract Base Class (ABC) aka Goose Typing**
Using the abstract base class is the proper way of implementing an interface in Python. A base class is the same as a super class, it is just a term from C++ and Python.   

Abstract means not implemented in this case.   

Let's implement the same BankAccount interface now using abstract base classes and dataclass

In [39]:
from dataclasses import dataclass, field
from typing import ClassVar, NamedTuple
from decimal import Decimal
from abc import ABC, abstractmethod


class BankAccount(ABC):
    '''an bankaccount interface'''
    
    @abstractmethod
    def deposit(self, amount:float)->None:
        pass
    
    @abstractmethod
    def withdrawl(self, amount:float)->None:
        pass
    
    @abstractmethod
    def transfer(self, counter:str, amount)->str:
        pass
    
    @abstractmethod
    def get_balance(self)->str:        
        pass
    
@dataclass
class CurrentAccount(BankAccount):
    account_nr:str
    account_holders:list[str] = field(default_factory=list) 
    balance:Decimal=Decimal('0')


In [40]:
cur = CurrentAccount(account_nr='NL01INSBEAU08642468',account_holders=['George','Rhino'])
cur

TypeError: Can't instantiate abstract class CurrentAccount with abstract methods deposit, get_balance, transfer, withdrawl

I have yet to implement all abstract methods. 
You cannot initialize a class without having implemented all the abstract methods inherited from its base class. This is an important thing to notice, an class with an abstract method can never be instantiated, and is an ABC itself.

In [39]:
from dataclasses import dataclass, field
from typing import ClassVar, NamedTuple
from decimal import Decimal
from abc import ABC, abstractmethod


class BankAccount(ABC):
    '''an bankaccount interface'''
    
    @abstractmethod
    def deposit(self, amount:float)->None:
        pass
    
    @abstractmethod
    def withdrawl(self, amount:float)->None:
        pass
    
    @abstractmethod
    def transfer(self, counter:str, amount)->str:
        pass
    
    @abstractmethod
    def get_balance(self)->str:        
        pass
    
@dataclass
class CurrentAccount(BankAccount):
    account_nr:str
    account_holders:list[str] = field(default_factory=list) 
    balance:Decimal=Decimal('0')
    
    def deposit(self, amount:float)->None:
        self.balance += Decimal(str(amount))
    
    def withdrawl(self, amount:float)->None:
        pass 
    
    def transfer(self, counter:str, amount)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            self.balance -= amount
            return f'the sum of {amount} euro has been transferred to account_nr {counter}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer'

    def get_balance(self)->str:        
        return f'this account has a balance of {self.balance} euro'

In [40]:
cur = CurrentAccount(account_nr='NL01INSBEAU08642468',account_holders=['George','Ente'])
cur

CurrentAccount(account_nr='NL01INSBEAU08642468', account_holders=['George', 'Ente'], balance=Decimal('0'))

In [41]:
cur.deposit(45000)
cur

CurrentAccount(account_nr='NL01INSBEAU08642468', account_holders=['George', 'Ente'], balance=Decimal('45000'))

In [42]:
cur.transfer(counter='NL57SNSB8765', amount=375.67)

'the sum of 375.67 euro has been transferred to account_nr NL57SNSB8765.'

In [43]:
cur.get_balance()

'this account has a balance of 44624.33 euro'

In [44]:
type(cur)

__main__.CurrentAccount

In [45]:
issubclass(CurrentAccount,BankAccount)

True

#### **Super()**
Even an abstract base class can implement methods and leave others abstract. This is useful for we can easily enough think of a use case where we want the base class to implement some methods for all its subclasses while letting the subclasses implement other themselves. 

This results in two questions:
 1. How do we make sure the base class’s implementation is used? Simple use a call to `super()` in the subclass implementation
 2. How can we make sure that a subclass does not override the base class implementation of the method. Again simple, you cannot. It is not Pythonic to do so, you must communicate why the implementation should not be overridden.


In [41]:
from dataclasses import dataclass, field
from typing import ClassVar, NamedTuple
from decimal import Decimal
from abc import ABC, abstractmethod


class BankAccount(ABC):
    '''an bankaccount interface'''
    
    @abstractmethod
    def deposit(self, amount:float)->None:
        pass
    
    @abstractmethod
    def withdrawl(self, amount:float)->None:
        pass
    
    @abstractmethod
    def transfer(self, counter:str, amount)->str:
        pass
    
    @abstractmethod
    def get_balance(self)->str:        
        pass
    
@dataclass
class CurrentAccount(BankAccount):
    account_nr:str
    account_holders:list[str] = field(default_factory=list) 
    balance:Decimal=Decimal('0')
    
    def deposit(self, amount:float)->None:
        self.balance += Decimal(str(amount))
    
    @abstractmethod
    def withdrawl(self, amount:float)->None:
        pass 
    
    def transfer(self, counter:str, amount)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            self.balance -= amount
            return f'the sum of {amount} euro has been transferred to account_nr {counter}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer'

    def get_balance(self)->str:        
        return f'this account has a balance of {self.balance} euro'
        
@dataclass
class ChildAccount(CurrentAccount):
    account_nr:str
    account_holders:list[str] = field(default_factory=list) 
    balance:Decimal=Decimal('0')
    
    def deposit(self, amount:float)->None:
        super().deposit(amount)
    
    def withdrawl(self, amount:float)->None:
        self.balance -= Decimal(str(amount))
        
    def transfer(self,counter:str,amount:float)->str:
        return super().transfer(counter, amount)
        
    def get_balance(self)->str:
        return super().get_balance()
        
    

In [42]:
child = ChildAccount(account_nr='NL63INGB09876', account_holders=['Ente'])
child.deposit(5000)
child.get_balance()

'this account has a balance of 5000 euro'

If you have complicated function you don't have to overwrite them. 
You can just implement the parent version, but it needs to be able to distinct which object the method is performed on.

In [43]:
child.transfer(counter='NL57SNSB876', amount=200)

'the sum of 200 euro has been transferred to account_nr NL57SNSB876.'

In [44]:
child.get_balance()

'this account has a balance of 4800 euro'

Let's expand the class a bit.

In [45]:
from dataclasses import dataclass, field
from typing import ClassVar, NamedTuple
from decimal import Decimal
from abc import ABC, abstractmethod


class BankAccount(ABC):
    '''an bankaccount interface'''
    
    @abstractmethod
    def deposit(self, amount:float)->None:
        pass
    
    @abstractmethod
    def withdrawl(self, amount:float)->None:
        pass
    
    @abstractmethod
    def transfer(self, counter:str, amount)->str:
        pass
    
    @abstractmethod
    def get_balance(self)->str:        
        pass
    
@dataclass
class CurrentAccount(BankAccount):
    account_nr:str
    account_holders:list[str] = field(default_factory=list) 
    balance:Decimal=Decimal('0')
    
    def deposit(self, amount:float)->None:
        self.balance += Decimal(str(amount))
    
    def withdrawl(self, amount:float)->None:
        self.balance -= Decimal(str(amount))
    
    def transfer(self, counter:str, amount)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            return f'the sum of {amount} euro has been transferred to account_nr {self.counter_account}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer'

    def get_balance(self)->str:        
        return f'this account has a balance of {self.balance} euro'
    
    #special method
    def __str__(self)->str:
        return f'the current account with number {self.account_nr} has a balance of {self.balance} euro.' 

@dataclass
class SavingsAccount(BankAccount):
    account_nr: str
    balance: Decimal = Decimal('0')
    rate:ClassVar[Decimal] = Decimal('2.0')
    counter_account: BankAccount = None
    account_holders:list[str] = field(default_factory = list)
    
    def deposit(self, amount:float)->None:
        self.balance = self.balance + Decimal(str(amount)) 
    
    def withdrawl(self, amount:float)->None:
        self.transfer(amount)
    
    def transfer(self,amount:float)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            self.balance = self.balance - amount
            return f'{amount} euro has been send to {self.counter_account} your account has been debited with {amount} euro'
        else:
            return f'you have insufficent funds on the account with number {self.account_number}, the current balance is {self.balance}' 
    
    def get_balance(self):
        return f'your savings are {self.balance} euro'
    
    @classmethod
    def set_interest_rate(self, rate=float)->None:
        self.rate = Decimal(str(rate))

    #special method    
    def __str__(self)->str:
        return f'the savings account with number {self.account_nr} has a balance of {self.balance} euro.' 
        
@dataclass
class InvestmentAccount(BankAccount):
    '''an investment account class'''
    account_nr:str
    counter_account:str
    account_holders:list[str]= field(default_factory=list)
    balance:Decimal = Decimal('0')
    stocks:list[(Stock,int,float)]=field(default_factory=list)
    funds:list[Fund]=field(default_factory=list)
    
    def deposit(self, amount:float)->None:
        self.balance += Decimal(str(amount))
    
    def withdrawl(self,amount):
        self.balance += Decimal(str(amount))
    
    def transfer(self, amount:float)->str:
        amount = Decimal(str(amount))
        if self.balance >= amount:
            return f'the sum of {amount} euro has been transferred to account_nr {self.counter_account}.'
        else:
            return f'account {self.account_nr} has insufficient funds for the requested transfer would you like to sell some stock or investment funds?'
    
    def get_balance(self)->str:
        return f'current balance is {self.balance} euro.'
    
    def buy_stock(self,stock:Stock, number:int)->str:
        if self.balance >= stock.current_price*number:
            self.balance -= stock.current_price*number
            self.stocks.append((Stock,number,stock.current_price))
            return f'we have added {number} of stock {stock.company} to your investment account at a price of {stock.current_price*number} euro.'
        else:
            return 'you have insufficient funds the transaction can`t be executed'
        
    def sell_stock(self,stock:str)->None:
        pass
    
    def invest_fund(self, fund:Fund, amount:float)->None:
        pass
    
    def disinvest_fund(self,fund_name:str)->None:
        pass
    
    #special methods
    def __str__(self)->str:
        return f'the investment account with number {self.account_nr} has a balance of {self.balance} euro.' 

    def __len__(self):
        '''This function doesn't make too much sense, but implementing your own function determing the size of a class does'''
        return sum([x for _,x,_ in self.stocks]) # uses pattern matching
        


In [46]:
cur = CurrentAccount(account_nr='NL01INSBEAU08642468',account_holders=['George','Rhino'])
cur.deposit(3800)
print(cur)

the current account with number NL01INSBEAU08642468 has a balance of 3800 euro.


In [47]:
sav = SavingsAccount(account_nr='NL57SNSB1234567', account_holders=['George','Croc'], counter_account='NL57SNSB0987654')
sav

SavingsAccount(account_nr='NL57SNSB1234567', balance=Decimal('0'), counter_account='NL57SNSB0987654', account_holders=['George', 'Croc'])

In [48]:
sav.deposit(78000)
print(sav)

the savings account with number NL57SNSB1234567 has a balance of 78000 euro.


In [49]:
inv = InvestmentAccount(account_nr='KEMPEN753197', account_holders=['Ente','Rhino'],counter_account='NL19INSBEAUF998999')
inv.deposit(25000000)
print(inv)

the investment account with number KEMPEN753197 has a balance of 25000000 euro.


In [50]:
inv.buy_stock(asml, 5000)

'we have added 5000 of stock ASML to your investment account at a price of 625000 euro.'

In [51]:
inv.buy_stock(microsoft, 10000)

'we have added 10000 of stock Microsoft to your investment account at a price of 2270000 euro.'

In [52]:
len(inv)

15000

**Using ABC is the proper way to implement an interface in Python!**

#### **Informal protocols**
Python really does not use the word interface. Instead, it uses protocols, which are sort of interfaces, and abstract base classes. Most likely you have implemented protocols without even knowing it, as many protocols in Python are used implicitly, which is kind of a Python super power.

For instance, we are implementing the sequence protocol in the example below.

In [53]:
class Vowels: 
    
    def __getitem__(self, i)->str:# this is unsafe implementation and will give you an index out of bound error
        return 'AEIOU'[i]
        

v = Vowels()
v[2]

'I'

`__getitem__()` is, as you know, a special method just implementing it is enough to implement the Sequence protocol    

In Python this is known as a dynamic protocol. We have only partially implemented the Sequence protocol. In actual fact, to completely implement the Sequence protocol you would have to implement 5 other methods.    

Just this partial implementation will allow you to do sequence things:
 1. index in 
 2. slice
 3. use de `in` operator
 4. iterate over it. 

In [54]:
v[1:3]

'EI'

In [55]:
'A' in v

True

In [56]:
for vowel in v:
    print(vowel)

A
E
I
O
U


This type of implicit protocol implementation is, as you can see, an immensely powerful tool in your Python toolkit.
It is one of the reasons Python is such a popular language. This type of partial on the fly implementation of a protocol is not possible in static languages, such as Java.

#### **static protocol**
Introduced in Python 3.8 there is also a static form of DuckTyping see https://peps.python.org/pep-0544/. A static protocol enforces you to implement the entire protocol, the main benefit is that you can use a type checker as MyPy https://www.mypy-lang.org/ to see if the protocol is completely implemented, and thus finally guarantee subtype.    

I am not going into static protocols for a quite simple reason. Static type checking by the interpreter in Python is not possible. Yes, you can static type checking with for instance MyPy, but this is at compile time at runtime Python does not care about MyPy. I then also feel that if you want to do static type checking, you might want to move to a language like Go-lang or if you don't mind the verbosity of Java (I do) use Java.    

Python's strength is that it is dynamic as we have seen in the previous example.


#### **Zope** 
Now I have mentioned static protocols, I should probably mention zope. It is a library that allows you to build actual interfaces in Python.
See https://zope.dev/

## **The end**